In [7]:
from azureml.core import Workspace
ws = Workspace.from_config() # 작업 공간의 셋팅 값을 가지고 올 수 있다
print('Workspace name:' + ws.name, 
    'Azure region:' + ws.location,
    'Subscription ID' + ws.subscription_id,
    'Resource Group:' + ws.resource_group
)

Workspace name:labuser2ml1 Azure region:eastus Subscription ID7ae06d59-97e1-4a36-bbfe-efb081b9b03b Resource Group:rg2


In [8]:
# 실험공간의 생성
from azureml.core import Experiment
experiment = Experiment(workspace = ws, name='diabets-experiment') # 워크스페이스를 지정을 해준다, experiment 변수에 저장을 시켜준다

In [9]:
from azureml.opendatasets import Diabetes # 당뇨병 데이터를 가지고 온다
from sklearn.model_selection import train_test_split # 정리하고 다듬을 꺼는 사이킷런을 가지고 온다, 그리고 데이터를 쪼개는 것을 가지고 온다

x_df = Diabetes.get_tabular_dataset().to_pandas_dataframe().dropna() # 애저에서 만든 pandas 데이터프레임을 가지고 온다, 빠져있는 데이터는 dropna 해준다
y_df = x_df.pop('Y') # 라벨 값을 가지고 오면서 x_df에는 Y를 없애준다(pop을 하였기 때문에 원본에는 Y의 값이 없는 것을 알 수 있다)

X_train, X_test, y_train, y_test = train_test_split(x_df, y_df, test_size = 0.2, random_state = 66) # 랜덤 시드를 정해준다
print(X_train)

In [6]:
# 훈련을 하기 위해서 몇가지 셋팅을 해둔다
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error # MSE를 메트릭으로 사용한다
from sklearn.externals import joblib # 작업을 할 때마다 저장을 해준다.
import math

# alpha값을 어떻게 주느냐에 따라 결과가 달라진다
alphas = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0] # 0.1부터 1까지 alpha값을 리스트로 만들어 준다

for alpha in alphas:
    run = experiment.start_logging() # 실험이 얼마나 돌았는지 기록을 남겨야 한다 그리고 기록에서 어떤 옵션을 남겼는지 본다
    run.log('alpha_value', alpha) # 기록을 남기는 방법, 그럼 alpha값이 남겨진다

    model = Ridge(alpha=alpha) # alpha의 값이 0.1 부터 1.0까지 반복해서 돌아간다
    model.fit(X_train,y_train) # train의 feature값과 label값을 넣어준다

    y_pred = model.predict(X_test) # 테스트 데이터를 주고 예측을 시켜준다
    rmse = math.sqrt(mean_squared_error(y_test,y_pred)) #  MSE에 루트를 씌워서 RMSE로 만들어 준다 (한꺼번에 두개의 값을 보기 위해서) 실제 값과 예측값을 rmse로 비교해본다
    run.log('rmse',rmse) # rmse를 기록해둔다
     # 여기까지 했으면 실험과 결과가 끝나게 된다
    model_name = 'model_alpha_' + str(alpha) + '.pkl' # alpha값이 몇으로 만들어 졌는지 보는 것이다 그리고 pkl의 확장자를 붙여준다
    filename = 'outputs/' + model_name # 반복하면서 테스트하면서 로그를 찍고 파일도 저장이 되며 output의 폴더 밑에 쭉 저장이 된다
    
    joblib.dump(value=model,filename=filename) # dump를 하면 저장을 하게 된다
    run.upload_file(name=model_name, path_or_stream=filename) # 파일로도 저장이 되어있고 애저에도 저장이 된다
    run.complete() # 작업이 끝난거를 명시를 해준다
    
    print(f'{alpha} experiment completed')

0.1 experiment completed
0.2 experiment completed
0.3 experiment completed
0.4 experiment completed
0.5 experiment completed
0.6 experiment completed
0.7 experiment completed
0.8 experiment completed
0.9 experiment completed
1.0 experiment completed
